In [6]:
import ee
ee.Initialize()
import numpy as np

# Make n random draws of parameter(s)
n = 3
rand_draws = list(np.random.normal(loc = 0, scale = 1, size = n))
print(rand_draws)

# Create imagecollection with n images
img = ee.Image.constant(1)
img_list = [img] * n
ic = ee.ImageCollection(img_list)

# Function to apply updating parameter
class img_transform:
    def __init__(self, rand_draws):
        self.rand_draws = rand_draws
    def __call__(self, img):
        rand_draw = self.rand_draws.pop()
        img = img.multiply(rand_draw)
        return img
    
# Initalize function
img_transformer = img_transform(rand_draws)

# Map function
ic = ic.map(img_transformer)

# Check updated values of images
point = {'type':'Point', 'coordinates':[ -116.88629,36.56122]}
ic.getRegion(point, n).getInfo()

[-0.38868796498228886, -1.4768968845568344, -0.41036997943879938]


[['id', 'longitude', 'latitude', 'time', 'constant'],
 ['0', -116.88628296156588, 36.56121197641992, None, -1.4768968845568344],
 ['1', -116.88628296156588, 36.56121197641992, None, -1.4768968845568344],
 ['2', -116.88628296156588, 36.56121197641992, None, -1.4768968845568344]]

In [10]:
import ee
ee.Initialize()
import numpy as np

# Make n random draws of parameter(s)
n = 3
rand_draws = list(np.random.normal(loc = 0, scale = 1, size = n))
print(rand_draws)

# Create imagecollection with n images
img = ee.Image.constant(1)
img_list = [img] * n

# Set metadata
new_img_list = []
for i, img in enumerate(img_list):
    img = img.set({'param': rand_draws[i]})
    new_img_list.append(img)
ic = ee.ImageCollection(new_img_list)
ic.getInfo()


[-1.334804577386618, 0.73149380351533688, 0.08582806214739068]


{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'max': 1,
      'min': 1,
      'precision': 'int',
      'type': 'PixelType'},
     'id': 'constant'}],
   'properties': {'param': -1.334804577386618, 'system:index': '0'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'max': 1,
      'min': 1,
      'precision': 'int',
      'type': 'PixelType'},
     'id': 'constant'}],
   'properties': {'param': 0.7314938035153369, 'system:index': '1'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'max': 1,
      'min': 1,
      'precision': 'int',
      'type': 'PixelType'},
     'id': 'constant'}],
   'properties': {'param': 0.08582806214739068, 'system:index': '2'},
   'type': 'Image'}],
 'type': 'ImageCollection'}

In [ ]:
import ee
ee.Initialize()
import numpy as np
import sys
sys.path.append('/rheil/notebooks/pygee')
import simulation

coefs_csv = '/rheil/notebooks/data/pooled_coefs.csv'
coefs_se_csv = '/rheil/notebooks/data/pooled_se.csv'

# Make n random draws of parameter(s)
n = 1000
draws = []
for i in range(n):
    draw = simulation.draw_random_coefs(coefs_csv, coefs_se_csv)
    draws.append(draw)

# Create imagecollection with n images
img = ee.Image.constant(1)
img_list = [img] * n

draw = draws[0]


# Set metadata
new_img_list = []
for i, img in enumerate(img_list):
    draw = draws[i]
    param_dict = draw.to_dict()[1]
    img = img.set(param_dict)
    new_img_list.append(img)
ic = ee.ImageCollection(new_img_list)
ic.getInfo()


In [21]:
import ee
ee.Initialize()
import numpy as np
import sys
sys.path.append('/rheil/notebooks/pygee')
import simulation

coefs_csv = '/rheil/notebooks/data/pooled_coefs.csv'
coefs_se_csv = '/rheil/notebooks/data/pooled_se.csv'

# Make n random draws of parameter(s)
n = 5
draws = []
for i in range(n):
    draw = simulation.draw_random_coefs(coefs_csv, coefs_se_csv)
    draws.append(draw)

# Create imagecollection with n images
img = ee.Image.constant(1)
img_list = [img] * n
ic = ee.ImageCollection(img_list)


In [16]:
draw_dict = {str(lu): ee.Dictionary(draw.to_dict()[lu]) for lu in draw.to_dict().keys()} 
draws_dict = {}
for i in range(n):
    draw = draws[i]
    draw_dict = {str(lu): ee.Dictionary(draw.to_dict()[lu]) for lu in draw.to_dict().keys()} 
    draws_dict[str(i)] = ee.Dictionary(draw_dict)
draws_dict = ee.Dictionary(draws_dict)

In [ ]:
img.get('system: index')

In [78]:
class sim_map:
    def __init__(self, coefs_csv, coefs_se_csv, n):
        self.coefs_csv = coefs_csv
        self.coefs_se_csv = coefs_se_csv
        draws_dict = {}
        for i in range(n):
            draw = simulation.draw_random_coefs(coefs_csv, coefs_se_csv)
            draw_dict = {str(lu): ee.Dictionary(draw.to_dict()[lu]) for lu in draw.to_dict().keys()} 
            draws_dict[str(i)] = ee.Dictionary(draw_dict)
        self.draws_dict = ee.Dictionary(draws_dict)
    def __call__(self, img):
        self.draw_dict = ee.Dictionary(self.draws_dict.get(ee.String('0')))
        self.lu_dict = ee.Dictionary(self.draw_dict.get(ee.String('1')))
        img = ee.Image(img.set(lu_dict))
        return img

In [79]:
sim_mapper = sim_map(coefs_csv, coefs_se_csv, n)
ic = ic.map(sim_mapper)

In [111]:
import ee
ee.Initialize()
import numpy as np

# Make n random draws of m parameter(s)
n = 1000
m = 2
rand_draws = list(np.random.normal(loc = 0, scale = 1, size = (n, m)))

# Convert to nested ee.Dictionary
param_dict = {i: ee.Dictionary({'param' + str(j): rand_draws[i][j] for j in range(m)}) for i in range(n)}
param_dict = ee.Dictionary(param_dict)
# print(param_dict)

# Create imagecollection with n images
img = ee.Image.constant(1)
img_list = [img] * n
ic = ee.ImageCollection(img_list)



# Function to apply updating parameter
def dict_to_eedict(python_dict):
    

class dict_to_params:
    def __init__(self, param_dict):
        self.param_dict = param_dict
    def __call__(self, img):
        i = img.get('system:index')
        params = self.param_dict.get(ee.String(i))
        img = img.set(params)
        return img
    
# Initalize function
d2p = dict_to_params(param_dict)

# # Map function
ic = ic.map(d2p)
ic.getInfo()

# # Check updated values of images
# point = {'type':'Point', 'coordinates':[ -116.88629,36.56122]}
# ic.getRegion(point, n).getInfo()

{'bands': [],
 'features': [{'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'max': 1,
      'min': 1,
      'precision': 'int',
      'type': 'PixelType'},
     'id': 'constant'}],
   'properties': {'param0': -0.5321505645918655,
    'param1': -1.2899003954742545,
    'system:index': '0'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'max': 1,
      'min': 1,
      'precision': 'int',
      'type': 'PixelType'},
     'id': 'constant'}],
   'properties': {'param0': -0.6100144817124515,
    'param1': 0.5505113833011165,
    'system:index': '1'},
   'type': 'Image'},
  {'bands': [{'crs': 'EPSG:4326',
     'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
     'data_type': {'max': 1,
      'min': 1,
      'precision': 'int',
      'type': 'PixelType'},
     'id': 'constant'}],
   'properties': {'param0': -0.04467606302974497,
    'param1': 0.20660998479

In [117]:
dict_test = {'key2': {'key_in3': 'value', 'key_in4': 'value'}, 'key1': {'key_in2': {'key_in_5': 'value'}, 'key_in1': 'value'}}
dict_test

{'key1': {'key_in1': 'value', 'key_in2': {'key_in_5': 'value'}},
 'key2': {'key_in3': 'value', 'key_in4': 'value'}}

In [124]:
any([isinstance(d, dict) for d in dict_test.values()])

True

In [143]:
def dict_depth(d, depth=0):
    if not isinstance(d, dict) or not d:
        return depth
    return max(dict_depth(v, depth+1) for v in d.values())

In [144]:
dict_depth(dict_test)

3

In [159]:
draw.index


Index(['plantrent', 'plantrent_luq2', 'plantrent_luq3', 'plantdum',
       'plantdum_luq2', 'plantdum_luq3', 'plantdum_south', 'plantdum_central',
       'plantdum_noprot', 'agrent', 'agrent_luq2', 'agrent_luq3', 'agdum',
       'agdum_luq2', 'agdum_luq3', 'agdum_south', 'agdum_central',
       'agdum_noprot', 'forrent', 'forrent_luq2', 'forrent_luq3', 'fordum',
       'fordum_luq2', 'fordum_luq3', 'fordum_south', 'fordum_central',
       'fordum_noprot'],
      dtype='object')

In [150]:
draw = simulation.draw_random_coefs(coefs_csv, coefs_se_csv)
draw_dict = draw.to_dict()
draws_dict = {'0': draw_dict, '1': draw_dict}
d = draws_dict

In [142]:
depth = dict_depth(d)
ee.Dictionary(ee.Dictionary({k: v for k, v in d.items})

NameError: name 'map_dicts' is not defined

In [210]:
ee_d = dict_to_eedict(d)
ee.Dictionary(ee_d.get('0')).get('1').getInfo()

{'agdum': '-3.30569209525',
 'agdum_central': '2.41870884122',
 'agdum_luq2': '-1.47442288114',
 'agdum_luq3': '-0.93686641513',
 'agdum_noprot': '0.842238777285',
 'agdum_south': '3.07887992834',
 'agrent': '0.00503440830809',
 'agrent_luq2': '-0.000386215405034',
 'agrent_luq3': '-0.0787266821927',
 'fordum': '0.792198643089',
 'fordum_central': '1.03750640733',
 'fordum_luq2': '2.17065397054',
 'fordum_luq3': '3.23497522419',
 'fordum_noprot': '-0.480741007636',
 'fordum_south': '-0.196499455317',
 'forrent': '0.00216915341046',
 'forrent_luq2': '-0.910472518259',
 'forrent_luq3': '-0.699182910191',
 'plantdum': '-2.69402786155',
 'plantdum_central': '2.27999541241',
 'plantdum_luq2': '-1.05331996194',
 'plantdum_luq3': '-10.562165665',
 'plantdum_noprot': '0.727397508652',
 'plantdum_south': '0.916232808352',
 'plantrent': '0.00488346681037',
 'plantrent_luq2': '1.23744440392',
 'plantrent_luq3': '5.97780295767'}

In [175]:
def gen_formulas_ee(coefs_df, lu_types=['ag', 'plant', 'for']):
    """
    Checks to make sure data_dict aligns with model coefficients from stata.
    Generates formulas for prediction generation.
    
    Parameters
    ----------
    coefs_df: pandas series
        Series containing all coefficients for conditional logit model
        
    Returns
    -------
    formula_list: list
        List containing text strings of formulas to execute for 
        simulation calculation
    
    data_dict: dict
        Updated data_dict including constants for dummy variables
    """
    coefs_lists = {'plant': ['plantrent', 'plantrent_luq2', 'plantrent_luq3', 'plantdum',
                             'plantdum_luq2', 'plantdum_luq3', 'plantdum_south', 'plantdum_central',
                             'plantdum_noprot'],
                   'ag': ['agrent', 'agrent_luq2', 'agrent_luq3', 'agdum',
                          'agdum_luq2', 'agdum_luq3', 'agdum_south', 'agdum_central',
                          'agdum_noprot'],
                   'for': ['forrent', 'forrent_luq2', 'forrent_luq3', 'fordum',
                           'fordum_luq2', 'fordum_luq3', 'fordum_south', 'fordum_central',
                           'fordum_noprot']}
    for k, coefs_list in coefs_lists.items(): 
        coefs_split = [var.split('_') for var in coefs_list]
        formula_dict = {}
        for lu_from in [1, 3, 5, 19]:
            formula_list=[]
            for var_list in coefs_split:
                formula=["b('" + var + "')" for var in var_list]
                formula=' * '.join(formula)
                formula+=" * img.get(('_'.join(var_list)) + '_' + str(lu_from))"
                formula_list.append(formula)
            formula_dict[lu_from] = sep_formulas(formula_list, lu_types)
    return formula_dict

def sep_formulas(formula_list, lu_types):
    """
    Parameters
    ----------
    formula_list: list
        List containing text strings of formulas to execute for 
        simulation calculation
    
    lu_types: list
        List of land use stubs to be found in formula_list
    
    Returns
    -------
    formula_dict: dict
        Dict containing full prediction formula for each land use type
        
    """
    formula_dict = ee.Dictionary()
    for lu_type in lu_types:
        l=['(' + formula + ')' for formula in formula_list if lu_type in formula]
        s=' + '.join(l)
        s = s.replace("b('" + lu_type + "dum')", '1.')
        formula_dict[lu_type] = s
    return formula_dict

In [176]:
coefs_df = draw
gen_formulas_ee(coefs_df)

TypeError: 'Dictionary' object does not support item assignment

In [170]:
formula_dict = simulation.gen_formulas_df(coefs_df)
formula_dict

{1: {'ag': "(b('agrent') * 0.00503440830809) + (b('agrent') * b('luq2') * -0.000386215405034) + (b('agrent') * b('luq3') * -0.0787266821927) + (1. * -3.30569209525) + (1. * b('luq2') * -1.47442288114) + (1. * b('luq3') * -0.93686641513) + (1. * b('south') * 3.07887992834) + (1. * b('central') * 2.41870884122) + (1. * b('noprot') * 0.842238777285)",
  'for': "(b('forrent') * 0.00216915341046) + (b('forrent') * b('luq2') * -0.910472518259) + (b('forrent') * b('luq3') * -0.699182910191) + (1. * 0.792198643089) + (1. * b('luq2') * 2.17065397054) + (1. * b('luq3') * 3.23497522419) + (1. * b('south') * -0.196499455317) + (1. * b('central') * 1.03750640733) + (1. * b('noprot') * -0.480741007636)",
  'plant': "(b('plantrent') * 0.00488346681037) + (b('plantrent') * b('luq2') * 1.23744440392) + (b('plantrent') * b('luq3') * 5.97780295767) + (1. * -2.69402786155) + (1. * b('luq2') * -1.05331996194) + (1. * b('luq3') * -10.562165665) + (1. * b('south') * 0.916232808352) + (1. * b('central') * 2.2

In [180]:
def sep_formulas(formula_list, lu_types):
    """
    Parameters
    ----------
    formula_list: list
        List containing text strings of formulas to execute for 
        simulation calculation
    
    lu_types: list
        List of land use stubs to be found in formula_list
    
    Returns
    -------
    formula_dict: dict
        Dict containing full prediction formula for each land use type
        
    """
    formula_dict={}
    for lu_type in lu_types:
        l=['(' + formula + ')' for formula in formula_list if lu_type in formula]
        s=' + '.join(l)
        s = s.replace("b('" + lu_type + "dum')", '1.')
        formula_dict[lu_type] = s
    return formula_dict


def gen_formulas_ee(coefs_df, lu_types=['ag', 'plant', 'for']):
    """
    Checks to make sure data_dict aligns with model coefficients from stata.
    Generates formulas for prediction generation.
    
    Parameters
    ----------
    coefs_df: pandas series
        Series containing all coefficients for conditional logit model
        
    Returns
    -------
    formula_list: list
        List containing text strings of formulas to execute for 
        simulation calculation
    
    data_dict: dict
        Updated data_dict including constants for dummy variables
    """
    coefs_list =  ['plantrent', 'plantrent_luq2', 'plantrent_luq3', 'plantdum',
                   'plantdum_luq2', 'plantdum_luq3', 'plantdum_south', 'plantdum_central',
                   'plantdum_noprot', 'agrent', 'agrent_luq2', 'agrent_luq3', 'agdum',
                   'agdum_luq2', 'agdum_luq3', 'agdum_south', 'agdum_central',
                   'agdum_noprot', 'forrent', 'forrent_luq2', 'forrent_luq3', 'fordum',
                   'fordum_luq2', 'fordum_luq3', 'fordum_south', 'fordum_central',
                   'fordum_noprot']
    coefs_split = [var.split('_') for var in coefs_list]
    formula_dict = {}
    for lu_from in [1, 3, 5, 19]:
        formula_list=[]
        for var_list in coefs_split:
            formula=["b('" + var + "')" for var in var_list]
            formula=' * '.join(formula)
            formula+=" * " + img.get(('_'.join(var_list)) + '_' + str(lu_from))
            formula_list.append(formula)
        formula_dict[lu_from] = sep_formulas(formula_list, lu_types)
    return formula_dict


def gen_formulas_df(coefs_df, lu_types=['ag', 'plant', 'for']):
    """
    Checks to make sure data_dict aligns with model coefficients from stata.
    Generates formulas for prediction generation.
    
    Parameters
    ----------
    coefs_df: pandas series
        Series containing all coefficients for conditional logit model
        
    Returns
    -------
    formula_list: list
        List containing text strings of formulas to execute for 
        simulation calculation
    
    data_dict: dict
        Updated data_dict including constants for dummy variables
    """
    coefs_list = [var.split('_') for var in coefs_df.index]
    formula_dict = {}
    for lu_from in [1, 3, 5, 19]:
        formula_list=[]
        for var_list in coefs_list:
            formula=["b('" + var + "')" for var in var_list]
            formula=' * '.join(formula)
            formula+=" * img.get(" + str(lu_from) + '_'.join(var_list) + '_' + ")"
            formula_list.append(formula)
        formula_dict[lu_from] = sep_formulas(formula_list, lu_types)
    return formula_dict

In [195]:
img = ee.Image.constant(1)
img = img.set({'param': 10})
img2 = ee.Image.constant(2)
img.getInfo()
# test = img.expression("img * param", {'img': img, 'param': img.get('param')})
test = img.multiply(ee.Number(img.get('param')))
test.getInfo()

{'bands': [{'crs': 'EPSG:4326',
   'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0],
   'data_type': {'max': 10,
    'min': 10,
    'precision': 'int',
    'type': 'PixelType'},
   'id': 'constant'}],
 'type': 'Image'}